In [1]:
# Install TensorFlow
!pip install tensorflow-gpu==2.0.0-beta1

     |████████████████████████████████| 348.9MB 61kB/s 
     |████████████████████████████████| 501kB 45.6MB/s 
     |████████████████████████████████| 3.1MB 46.2MB/s 


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np

In [3]:
# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

169009152/169001437 [==============================] - 3s 0us/step


In [4]:
# x_dataset = np.concatenate((x_train, x_test)).reshape((x_train.shape[0] + x_test.shape[0], x_test.shape[1], x_test.shape[2],1))
x_dataset = np.concatenate((x_train, x_test))
y_dataset = np.concatenate((y_train, y_test))
print(x_train.shape, x_test.shape, x_dataset.shape)
print(y_train.shape, y_test.shape, y_dataset.shape)

(50000, 32, 32, 3) (10000, 32, 32, 3) (60000, 32, 32, 3)
(50000, 1) (10000, 1) (60000, 1)


In [5]:
new_y_dataset = np.zeros((y_dataset.shape[0], np.max(y_dataset) + 1), dtype=x_train.dtype)

for i in range(0, y_dataset.shape[0]):
    new_y_dataset[i][y_dataset[i]] = 1
    
print(y_dataset.shape, new_y_dataset.shape, y_dataset[0])
print(new_y_dataset[0])

(60000, 1) (60000, 100) [19]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [6]:
split_x_dataset = np.split(x_dataset, 2)
split_y_dataset = np.split(new_y_dataset, 2)

x_training_half = split_x_dataset[0]
y_training_half = split_y_dataset[0]

x_testing_half = split_x_dataset[1]
y_testing_half = split_y_dataset[1]

print(x_training_half.shape, y_training_half.shape)
print(x_testing_half.shape, y_testing_half.shape)

(30000, 32, 32, 3) (30000, 100)
(30000, 32, 32, 3) (30000, 100)


In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), input_shape=(x_dataset.shape[1], x_dataset.shape[2], x_dataset.shape[3]), padding="same"),
    tf.keras.layers.Conv2D(64, (3, 3), padding="valid"),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(new_y_dataset.shape[1], activation='softmax')
])
model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 14400)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               7373312   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 100)               2

In [8]:
model.fit(x_training_half, y_training_half, epochs=15)

W0718 15:03:04.625765 140524847171456 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 30000 samples
Epoch 1/15
30000/30000 [==============================] - 10s 340us/sample - loss: 3.7628 - accuracy: 0.1344
Epoch 2/15
30000/30000 [==============================] - 6s 216us/sample - loss: 2.9105 - accuracy: 0.2783
Epoch 3/15
30000/30000 [==============================] - 6s 215us/sample - loss: 2.2258 - accuracy: 0.4156
Epoch 4/15
30000/30000 [==============================] - 7s 218us/sample - loss: 1.5626 - accuracy: 0.5668
Epoch 5/15
30000/30000 [==============================] - 6s 216us/sample - loss: 0.9830 - accuracy: 0.7128
Epoch 6/15
30000/30000 [==============================] - 6s 216us/sample - loss: 0.6427 - accuracy: 0.8112
Epoch 7/15
30000/30000 [==============================] - 6s 217us/sample - loss: 0.4903 - accuracy: 0.8533
Epoch 8/15
30000/30000 [==============================] - 6s 215us/sample - loss: 0.4307 - accuracy: 0.8697
Epoch 9/15
30000/30000 [==============================] - 7s 220us/sample - loss: 0.3821 - accuracy: 0.8870
Epoc

In [9]:
model.evaluate(x_testing_half, y_testing_half)

30000/30000 [==============================] - 3s 93us/sample - loss: 7.6860 - accuracy: 0.2626


[7.685996899414063, 0.26263332]

In [0]:
predict_train = model.predict(x_training_half)
predict_test = model.predict(x_testing_half)

In [11]:
c = 0
for i in range(0, len(x_training_half)):
    if np.argmax(predict_train[i]) == np.argmax(y_training_half[i]):
        c += 1
print(c * 100 / len(x_training_half))

93.74666666666667


In [12]:
c = 0
for i in range(0, len(x_testing_half)):
    if np.argmax(predict_train[i]) == np.argmax(y_testing_half[i]):
        c += 1
print(c * 100 / len(x_testing_half))

1.0533333333333332
